In [20]:
import torch
from torch import nn
import torchlibrosa
import numpy as np
import onnx_coreml


class WaveToSpectrogram(nn.Module):

    def __init__( self, n_fft, hop_length, center=False ):
        super(WaveToSpectrogram, self).__init__()

        self.spec_extractor = torchlibrosa.stft.Spectrogram(
            n_fft=n_fft,
            hop_length=hop_length,
            center=center,
        )

    def forward( self, x ):
        return self.spec_extractor( x )
    
    def convert_to_onnx( self, filename_onnx, sample_input ):
        input_names = [ 'input.1' ]
        output_names = ['Spectrogram']

        torch.onnx.export(
            self,
            torch.from_numpy( sample_input ),
            filename_onnx,
            input_names=input_names,
            output_names=output_names,                     
        )

In [22]:
# load in waveform
import soundfile
waveform, samplerate = soundfile.read( '../urbansound8k/audio/fold1/101415-3-0-2.wav' )
sample_input = waveform[:192000].astype( dtype=np.float32).reshape((1, 192000))


In [19]:
model = WaveToSpectrogram( n_fft=1024, hop_length=512 )
torch.save(model, 'Hello.pt')


/Users/hugo/opt/anaconda3/envs/audio/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)


## Convert to Onnx

In [24]:


# instantiate model and export it
model = WaveToSpectrogram( n_fft=1024, hop_length=512 )

model.convert_to_onnx( 'wave_to_spectrogram_model.onnx', sample_input )

## Convert to Coreml

In [25]:
import onnx_coreml

mlmodel = onnx_coreml.convert(
    model = 'wave_to_spectrogram_model.onnx',
    predicted_feature_name = [],
    minimum_ios_deployment_target='13',    
)
mlmodel.save( '/wave_to_spec.mlmodel' )

NameError: name 'onnx_coreml' is not defined